# Mutual information calculation

In [ ]:
import pandas as pd

from scipy import stats
from scipy.stats import mannwhitneyu as mw
from scipy.stats import sem
from scipy.stats import iqr
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from scipy.stats import wilcoxon

import random

import math

import EntropyHub as EH

import numpy as np

import time

from os import listdir
from os.path import isfile, join

import neo

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
raw_data = pd.read_excel("Nikita_16_11_22_updrs.xlsx") #original file with other SUA params

# patients = list(raw_data["patient"].unique())

# params = list(raw_data)[14:]
# for param in params:
#     raw_data[param].fillna(raw_data[param].median(), inplace=True)

In [ ]:
params = ['sib', 'isitm', 'isitisi']

In [ ]:
files_list = []
max_wl = 7
for i in range(2, max_wl + 1):
    files_list.append("encoded_data_with_ents_wl" + str(i) + ".csv") # open files created with 3 ENCODED ISI


In [ ]:
for i in range(len(files_list)):
    if i == 0:
        df = pd.read_csv(files_list[i])
        result = pd.merge(raw_data, df, on=["Unnamed: 0.1.1"])
        print(files_list[i])
    #         result = result.drop(['interval_name_y'], axis=1)
    else:
        df = pd.read_csv(files_list[i])
        result = pd.merge(result, df, on=["Unnamed: 0.1.1"])
        print(files_list[i])
#         result = result.drop(['interval_name_y'], axis=1)


In [ ]:
for el in list(result):
    if "_x" in el or "_y" in el or el in params:
        try:
            result = result.drop(columns=el)
        except:
            pass


In [ ]:
for i in range(1, 8):
    for el in ["isitisi", "isitm", "sib"]:
        MI = "mut_inf_" + el + "_wl" + str(i)  # mutual information I(X;Y)
        X_ent = "ent_" + el + "_wl" + str(i)
        cond_XY = "cond_ent_" + el + "_wl" + str(i)
        result[MI] = result[X_ent] - result[cond_XY]

        diff_X_Y = "ent_diff_" + el + "_wl" + str(i)  # entropies difference H(X)-H(Y)
        Y_ent = "shuf_ent_" + el + "_wl" + str(i)
        result[diff_X_Y] = result[X_ent] - result[Y_ent]

        joint_X_Y = "j_ent_" + el + "_wl" + str(i)  # joint entropy H(XY)
        result[joint_X_Y] = result[X_ent] + result[Y_ent] - result[MI]

        ar_ent = "ar_ent_" + el + "_wl" + str(i)  # H' = max[H]-H(XY)
        result[ar_ent] = i**2 - result[joint_X_Y]

groups = result.groupby("STN_ID")
rel_depth_list = []
for name, group in groups:
    min_depth = min(group["depth"])
    temp_depth = group["depth"] - min_depth
    max_depth = max(temp_depth)
    rel_depth_list += list(temp_depth / max_depth)
result["norm_depth"] = rel_depth_list
result["updrs_diff"] = result["updrs_off"] - result["updrs_on"]


In [ ]:
result.to_csv('file_to_analyse_cond.csv')

In [ ]:
result.to_excel('file_to_analyse_cond.xlsx')